In [13]:
import pandas as pd
import numpy as np

import flwr as fl
from flwr.common import Metrics

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

from collections import OrderedDict
from typing import List, Tuple
import sys
import time
import warnings
warnings.simplefilter('ignore')

In [14]:
NUM_CLIENTS = 3
EPOCHS = 10
ROUNDS = 5

In [31]:
def load_dataset():
    df = pd.read_csv('../datasets/label_data.csv')
    df = df.rename(columns={'label': 'target'})
    X = df.iloc[:, :-1]
    y = df.loc[:, 'target']
    
    # Split the training set and testing set in 80% ratio
    X_train, X_test, y_train, y_test = (
        train_test_split(
            X, y, 
            train_size=0.8,
            random_state=42, 
            shuffle=True,
            stratify=y,
        )
    )
    
    # Split the training set by the number of clients
    X_train = np.array_split(X_train, NUM_CLIENTS)
    y_train = np.array_split(y_train, NUM_CLIENTS)
    
    return X_train, X_test, y_train, y_test
    
X_train, X_test, y_train, y_test = load_dataset()

In [20]:
# Create RandomForestClassifier model
model = RandomForestClassifier(
    class_weight='balanced',
    warm_start=True,
)

In [22]:
def get_model_params(model):

In [22]:
def set_model_params(model: RandomForestClassifier, params: RFParams) -> RandomForestClassifier:
    

In [23]:
def params_initialization(model: RandomForestClassifier):
    

In [23]:
class FlowerClient(fl.client.NumPyClient):
    
    def get_params(self):
        return get_model_params(model)
    
    def fit(self, params, config):
        set_model_params(model, params)
    
        model.fit(X_train, y_train)
        print(f"Training finished for round {config['rnd']}.")
        return get_model_params(model), len(X_train), {}
    
    def evaluate(self, params, config):
        set_model_params(model, params)
        loss = 
        accuracy = 
        
        return float(loss), len(X_test), {'accuracy: ': float(accuracy)}

In [ ]:
model = RandomForestClassifier()
params_initialization(model)

strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=1.0,
    min_fit_clients=NUM_CLIENTS,
    min_evaluate_clients=NUM_CLIENTS,
    min_available_clients=NUM_CLIENTS,

)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=ROUNDS),
    strategy=strategy,
)